#### **Passo 1 - Importar arquivos e bibliotecas.**

In [1]:
# Importar bibliotecas.
import pandas as pd
from pathlib import Path
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# Definir caminhos.
caminho_backup = Path('../Backup Arquivos Lojas')
caminho_bases = Path('../Bases de Dados')

In [2]:
# Importar dia_indicadorFrames.
lojas_df = pd.read_csv(f'{caminho_bases}\Lojas.csv', sep=';', encoding='latin1')
vendas_df = pd.read_excel(f'{caminho_bases}\Vendas.xlsx')
emails_df = pd.read_excel(f'{caminho_bases}\Emails.xlsx')

#### **Passo 2 - Criar tabelas de vendas separadas para cada loja.**

In [ ]:
# Juntar o nome das lojas ao vendas_df.
vendas_df = vendas_df.merge(lojas_df, on='ID Loja')
display(vendas_df.head(3))

In [4]:
# Separar as vendas de cada loja em um dicionário.
dicionario_lojas = {}
for loja in lojas_df['Loja']: dicionario_lojas[loja] = vendas_df.loc[vendas_df['Loja'] == loja, :]

In [5]:
# Pegar a última data do registro.
dia_indicador = vendas_df['Data'].max()
print(dia_indicador)
print(dia_indicador.day, dia_indicador.month, dia_indicador.year)

2019-12-26 00:00:00
26 12 2019


#### **Passo 3 - Salvar planilhas na pasta Backup.**

In [6]:
if not caminho_backup.exists(): # Se a pasta não existir.
    os.mkdir(caminho_backup) # Cria a pasta.

# Lista de arquivos/diretório na pasta backup atual.
lista_arquivos_backup = [arquivo.name for arquivo in caminho_backup.iterdir()]

# Criar pastas e salvar o arquivo para cada loja.
for loja in dicionario_lojas:
    if not loja in lista_arquivos_backup: # Se não tiver pasta para uma loja.
        os.mkdir(caminho_backup / Path(loja)) # Criar a pasta.
    else: # Se já existir a pasta.
        nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
        local_arquivo = caminho_backup / Path(loja) / Path(nome_arquivo)
        dicionario_lojas[loja].to_excel(local_arquivo)

#### **Passo 4 - Calcular os indicadores para 1 loja.**

In [7]:
loja = 'Norte Shopping'
vendas_loja = dicionario_lojas[loja]
vendas_loja_dia = vendas_loja.loc[vendas_loja['Data'] == dia_indicador, :]

# Faturamento.
faturamento_ano = vendas_loja['Valor Final'].sum()
faturamento_dia = vendas_loja_dia['Valor Final'].sum()

# Diversidade de produtos.
qtde_produtos_ano = len(vendas_loja['Produto'].unique())
qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

# Ticket medio == Valor médio das compras.
valor_venda = vendas_loja[['Código Venda', 'ID Loja', 'Quantidade', 'Valor Unitário', 'Valor Final']].groupby('Código Venda').sum()
ticket_medio_ano = valor_venda['Valor Final'].mean()
valor_venda_dia = vendas_loja_dia[['Código Venda', 'ID Loja', 'Quantidade', 'Valor Unitário', 'Valor Final']].groupby('ID Loja').sum()
ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
print(ticket_medio_ano)
print(ticket_medio_dia)

784.5866177818515
1259.0


In [8]:
# Definir metas.
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

#### **Passo 5 - Enviar email para 1 gerente.**

##### Alterar os e-mails para conseguir enviar para mim mesmo.

##### Corpo do e-mail.

In [ ]:
nome = emails_df.loc[emails_df['Loja'] == loja, 'Gerente'].values[0]

cor_fat_dia = 'green' if faturamento_dia >= meta_faturamento_dia else 'red'
cor_fat_ano = 'green' if faturamento_ano >= meta_faturamento_ano else 'red'
cor_qtde_dia = 'green' if qtde_produtos_dia >= meta_qtdeprodutos_dia else 'red'
cor_qtde_ano = 'green' if qtde_produtos_ano >= meta_qtdeprodutos_ano else 'red'
cor_ticket_dia = 'green' if ticket_medio_dia >= meta_ticketmedio_dia else 'red'
cor_ticket_ano = 'green' if ticket_medio_ano >= meta_ticketmedio_ano else 'red'

corpo_email = f'''
<p>Bom dia, {nome}</p>

<p>O resultado de ontem <strong>({dia_indicador.month}/{dia_indicador.day})</strong> da <strong>Loja {loja}</strong> foi:</p>

<table>
    <tr>
        <th style="padding: 10px;">Indicador</th>
        <th style="padding: 10px;">Valor Dia</th>
        <th style="padding: 10px;">Meta Dia</th>
        <th style="padding: 10px;">Cenário Dia</th>
    </tr>
    <tr>
        <td style="text-align: left; padding: 0px 10px 0px 0px;">Faturamento</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;">R${faturamento_dia:_.2f}</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;">R${meta_faturamento_dia:_.2f}</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;"><font color='{cor_fat_dia}'>◙</font></td>
    </tr>
    <tr>
        <td style="text-align: left; padding: 0px 10px 0px 0px;">Diversidade de Produtos</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;">{qtde_produtos_dia}</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;">{meta_qtdeprodutos_dia}</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;"><font color='{cor_qtde_dia}'>◙</td>
    </tr>
    <tr>
        <td style="text-align: left; padding: 0px 10px 0px 0px;">Ticket Médio</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;">R${ticket_medio_dia:_.2f}</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;">R${meta_ticketmedio_dia:_.2f}</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;"><font color='{cor_ticket_dia}'>◙</td>
    </tr>
</table>

<br>

<table>
    <tr>
        <th style="padding: 10px;">Indicador</th>
        <th style="padding: 10px;">Valor Ano</th>
        <th style="padding: 10px;">Meta Ano</th>
        <th style="padding: 10px;">Cenário Ano</th>
    </tr>
    <tr>
        <td style="text-align: left; padding: 0px 10px 0px 0px;">Faturamento</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;">R${faturamento_ano:_.2f}</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;">R${meta_faturamento_ano:_.2f}</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;"><font color='{cor_fat_ano}'>◙</font></td>
    </tr>
    <tr>
        <td style="text-align: left; padding: 0px 10px 0px 0px;">Diversidade de Produtos</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;">{qtde_produtos_ano}</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;">{meta_qtdeprodutos_ano}</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;"><font color='{cor_qtde_ano}'>◙</td>
    </tr>
    <tr>
        <td style="text-align: left; padding: 0px 10px 0px 0px;">Ticket Médio</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;">R${ticket_medio_ano:_.2f}</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;">R${meta_ticketmedio_ano:_.2f}</td>
        <td style="text-align: center; padding: 0px 10px 0px 0px;"><font color='{cor_ticket_ano}'>◙</td>
    </tr>
</table>

<br>

<p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

<p>Qualquer dúvida estou a disposição.</p>
<p>Att., Funcionario</p>
'''.replace('.', ',').replace('_','.')

##### Enviar e-mail.

In [90]:
# Configurações do servidor e e-mail remetente
email_remetente = 'email_Funcionario@gmail.com'
senha_remetente = 'xxxxxxxxx'          # Use a senha de aplicativo gerada anteriormente.
smtp_host = "smtp.gmail.com"
smtp_port = 587

# Destinatário e assunto do e-mail
email_destinatario = emails_df.loc[emails_df['Loja'] == loja, 'E-mail'].values[0]
assunto = f"OnePage Dia {dia_indicador.strftime('%d/%m/%Y')} - Loja {loja}"

# Criação da mensagem
msg = MIMEMultipart()
msg["From"] = email_remetente
msg["To"] = email_destinatario
msg["Subject"] = assunto

# Adiciona o corpo do e-mail em HTML
msg.attach(MIMEText(corpo_email, "html"))

# Arquivo anexar.
arquivo = Path(f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx')
caminho_arquivo = caminho_backup / loja / arquivo
part = MIMEBase("application", "octet-stream")
with open(caminho_arquivo, "rb") as file:
    part.set_payload(file.read())
encoders.encode_base64(part)
part.add_header("Content-Disposition", f"attachment; filename= {arquivo}")
msg.attach(part)

# Conexão com o servidor SMTP
try:
    server = smtplib.SMTP(smtp_host, smtp_port)
    server.starttls()
    server.login(email_remetente, senha_remetente)
    
    # Envio do e-mail
    server.sendmail(email_remetente, email_destinatario, msg.as_string())
    print("E-mail enviado com sucesso!")
except Exception as e:
    print(f"Erro ao enviar o e-mail: {e}")
finally:
    server.quit()

E-mail enviado com sucesso!


#### **Passo 5 - Fazer para todos as lojas.**

In [6]:
# Definir metas.
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

In [ ]:
for loja in dicionario_lojas:
    ### Calcular indicadores para loja. ###

    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data'] == dia_indicador, :]

    # Faturamento.
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    # Diversidade de produtos.
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    # Ticket medio == Valor médio das compras.
    valor_venda = vendas_loja[['Código Venda', 'ID Loja', 'Quantidade', 'Valor Unitário', 'Valor Final']].groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    valor_venda_dia = vendas_loja_dia[['Código Venda', 'ID Loja', 'Quantidade', 'Valor Unitário', 'Valor Final']].groupby('ID Loja').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    
    ### Corpo do e-mail. ###
    
    nome = emails_df.loc[emails_df['Loja'] == loja, 'Gerente'].values[0]

    cor_fat_dia = 'green' if faturamento_dia >= meta_faturamento_dia else 'red'
    cor_fat_ano = 'green' if faturamento_ano >= meta_faturamento_ano else 'red'
    cor_qtde_dia = 'green' if qtde_produtos_dia >= meta_qtdeprodutos_dia else 'red'
    cor_qtde_ano = 'green' if qtde_produtos_ano >= meta_qtdeprodutos_ano else 'red'
    cor_ticket_dia = 'green' if ticket_medio_dia >= meta_ticketmedio_dia else 'red'
    cor_ticket_ano = 'green' if ticket_medio_ano >= meta_ticketmedio_ano else 'red'

    corpo_email = f'''
    <p>Bom dia, {nome}</p>

    <p>O resultado de ontem <strong>({dia_indicador.month}/{dia_indicador.day})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <table>
        <tr>
            <th style="padding: 10px;">Indicador</th>
            <th style="padding: 10px;">Valor Dia</th>
            <th style="padding: 10px;">Meta Dia</th>
            <th style="padding: 10px;">Cenário Dia</th>
        </tr>
        <tr>
            <td style="text-align: left; padding: 0px 10px 0px 0px;">Faturamento</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;">R${faturamento_dia:_.2f}</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;">R${meta_faturamento_dia:_.2f}</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;"><font color='{cor_fat_dia}'>◙</font></td>
        </tr>
        <tr>
            <td style="text-align: left; padding: 0px 10px 0px 0px;">Diversidade de Produtos</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;">{qtde_produtos_dia}</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;">{meta_qtdeprodutos_dia}</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;"><font color='{cor_qtde_dia}'>◙</td>
        </tr>
        <tr>
            <td style="text-align: left; padding: 0px 10px 0px 0px;">Ticket Médio</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;">R${ticket_medio_dia:_.2f}</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;">R${meta_ticketmedio_dia:_.2f}</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;"><font color='{cor_ticket_dia}'>◙</td>
        </tr>
    </table>

    <br>

    <table>
        <tr>
            <th style="padding: 10px;">Indicador</th>
            <th style="padding: 10px;">Valor Ano</th>
            <th style="padding: 10px;">Meta Ano</th>
            <th style="padding: 10px;">Cenário Ano</th>
        </tr>
        <tr>
            <td style="text-align: left; padding: 0px 10px 0px 0px;">Faturamento</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;">R${faturamento_ano:_.2f}</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;">R${meta_faturamento_ano:_.2f}</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;"><font color='{cor_fat_ano}'>◙</font></td>
        </tr>
        <tr>
            <td style="text-align: left; padding: 0px 10px 0px 0px;">Diversidade de Produtos</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;">{qtde_produtos_ano}</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;">{meta_qtdeprodutos_ano}</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;"><font color='{cor_qtde_ano}'>◙</td>
        </tr>
        <tr>
            <td style="text-align: left; padding: 0px 10px 0px 0px;">Ticket Médio</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;">R${ticket_medio_ano:_.2f}</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;">R${meta_ticketmedio_ano:_.2f}</td>
            <td style="text-align: center; padding: 0px 10px 0px 0px;"><font color='{cor_ticket_ano}'>◙</td>
        </tr>
    </table>

    <br>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou a disposição.</p>
    <p>Att., João</p>
    '''.replace('.', ',').replace('_','.')

    ### Enviar e-mail. ###

    # Configurações do servidor e e-mail remetente
    email_remetente = 'email_funcionario@gmail.com'
    senha_remetente = 'xxxxxxxxx'          # Use a senha de aplicativo gerada anteriormente.
    smtp_host = "smtp.gmail.com"
    smtp_port = 587

    # Destinatário e assunto do e-mail
    email_destinatario = emails_df.loc[emails_df['Loja'] == loja, 'E-mail'].values[0]
    assunto = f"OnePage Dia {dia_indicador.strftime('%d/%m/%Y')} - Loja {loja}"

    # Criação da mensagem
    msg = MIMEMultipart()
    msg["From"] = email_remetente
    msg["To"] = email_destinatario
    msg["Subject"] = assunto

    # Adiciona o corpo do e-mail em HTML
    msg.attach(MIMEText(corpo_email, "html"))

    # Arquivo anexar.
    arquivo = Path(f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx')
    caminho_arquivo = caminho_backup / loja / arquivo
    part = MIMEBase("application", "octet-stream")
    with open(caminho_arquivo, "rb") as file:
        part.set_payload(file.read())
    encoders.encode_base64(part)
    part.add_header("Content-Disposition", f"attachment; filename= {arquivo}")
    msg.attach(part)

    # Conexão com o servidor SMTP
    try:
        server = smtplib.SMTP(smtp_host, smtp_port)
        server.starttls()
        server.login(email_remetente, senha_remetente)
        
        # Envio do e-mail
        server.sendmail(email_remetente, email_destinatario, msg.as_string())
        print(f"E-mail da Loja {loja} enviado com sucesso!")
    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")
    finally:
        server.quit()

#### **Passo 6 - Criar ranking para a diretoria.**

In [10]:
# Faturamento no ano.
faturamento_lojas_ano = vendas_df.groupby('Loja')[['Valor Final']].sum()
faturamento_lojas_ano.sort_values('Valor Final', ascending=False, inplace=True)

# Salvar arquivo.
arquivo = f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
local_arquivo = caminho_backup / Path(arquivo)
faturamento_lojas_ano.to_excel(local_arquivo)

# Faturamento no dia.
vendas_dia = vendas_df.loc[vendas_df['Data'] == dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Valor Final']].sum()
faturamento_lojas_dia.sort_values('Valor Final', ascending=False, inplace=True)

# Salvar arquivo.
arquivo = f'{dia_indicador.month}_{dia_indicador.day}_Ranking Dia.xlsx'
local_arquivo = caminho_backup / Path(arquivo)
faturamento_lojas_dia.to_excel(local_arquivo)

#### **Passo 7 - Enviar e-mail para a diretoria com o Ranking.**

In [28]:
corpo_email = f'''
Prezados, bom dia

Melhor loja do Dia em faturamento: Loja {faturamento_lojas_dia.index[0]} com Faturamento R${faturamento_lojas_dia['Valor Final'][0]:_.2f}
Pior loja do Dia em faturamento: Loja {faturamento_lojas_dia.index[-1]} com Faturamento R${faturamento_lojas_dia['Valor Final'][-1]:_.2f}

Melhor loja do Ano em faturamento: Loja {faturamento_lojas_ano.index[0]} com Faturamento R${faturamento_lojas_ano['Valor Final'][0]:_.2f}
Pior loja do Ano em faturamento: Loja {faturamento_lojas_ano.index[0]} com Faturamento R${faturamento_lojas_ano['Valor Final'][-1]:_.2f}

Segue em anexo os rankings do ano e dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att., João
'''.replace('.',',').replace('_','.')

In [29]:
# Configurações do servidor e e-mail remetente
email_remetente = 'email_funcionario@gmail.com'
senha_remetente = 'xxxxxxxxx'          # Use a senha de aplicativo gerada anteriormente.
smtp_host = "smtp.gmail.com"
smtp_port = 587

# Destinatário e assunto do e-mail
email_destinatario = emails_df.loc[emails_df['Loja'] == 'Diretoria', 'E-mail'].values[0]
assunto = f"Ranking Dia {dia_indicador.strftime('%d/%m/%Y')}"

# Criação da mensagem
msg = MIMEMultipart()
msg["From"] = email_remetente
msg["To"] = email_destinatario
msg["Subject"] = assunto

# Adiciona o corpo do e-mail
msg.attach(MIMEText(corpo_email, "plain"))

# Lista de arquivos a serem anexados
arquivos_anexo = [f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx', 
                  f'{dia_indicador.month}_{dia_indicador.day}_Ranking Dia.xlsx']

# Anexando os arquivos à mensagem
for arquivo in arquivos_anexo:
    part = MIMEBase("application", "octet-stream")
    with open(caminho_backup / arquivo, "rb") as file:
        part.set_payload(file.read())
    encoders.encode_base64(part)
    part.add_header("Content-Disposition", f"attachment; filename= {arquivo}")
    msg.attach(part) 


# Conexão com o servidor SMTP
try:
    server = smtplib.SMTP(smtp_host, smtp_port)
    server.starttls()
    server.login(email_remetente, senha_remetente)
    
    # Envio do e-mail
    server.sendmail(email_remetente, email_destinatario, msg.as_string())
    print("E-mail da Diretoria enviado com sucesso!")
except Exception as e:
    print(f"Erro ao enviar o e-mail: {e}")
finally:
    server.quit()

E-mail da Diretoria enviado com sucesso!
